In [6]:
# Description : Working of One of the Popular Trading Indicator "Alligator Indicator" using python and plot in plotly.

In [1]:
# Import Python Libraries
import pandas as pd
import numpy as np
import pandas_datareader as web
import matplotlib.pyplot as plt

plt.style.use('fivethirtyeight')

import plotly.graph_objects as go

In [11]:
# Get the Stock Symbol for the Portfolio 
stockSymbols = ['GAIL.NS']

# GET the stock starting date
stockDate = '2020-01-01'

# Get the stock ending date.
stock_End_date = '2020-09-30'

In [12]:
# Create a function to get the stock prices in the portfolio
def getMyPortfolio(stocks = stockSymbols, start = stockDate, end=stock_End_date):
  data = web.DataReader(stocks, data_source='yahoo', start=start, end=end, )
  data.columns = ['AdjClose', 'Close', 'High', 'Low', 'Open', 'Volume']
  return data

In [13]:
#  Get the stock portfolio Adj. Close price
my_stocks = getMyPortfolio(stockSymbols)
my_stocks

,AdjClose,Close,High,Low,Open,Volume
Date,,,,,,
2020-01-01,114.870499,121.199997,121.800003,119.750000,121.400002,4704499.0
2020-01-02,117.097778,123.550003,123.949997,120.650002,121.500000,6747917.0
2020-01-03,119.088112,125.650002,128.800003,123.699997,123.699997,21625485.0
2020-01-06,117.855995,124.349998,126.250000,122.849998,125.000000,7719845.0
2020-01-07,117.950775,124.449997,126.050003,123.849998,124.900002,9298669.0
...,...,...,...,...,...,...
2020-09-25,85.199997,85.199997,85.599998,83.099998,83.699997,13060155.0
2020-09-28,88.199997,88.199997,89.099998,85.550003,86.550003,12545459.0
2020-09-29,87.500000,87.500000,88.800003,87.000000,88.800003,9090406.0


# Alligator Indicator : 
- The Alligator Indicator is most popular amongs the stock analizers:
- This Alligator Indicator help to analyze the stocks or indicate the current stock position, performace.
- This Alligator indicator is combination of Three *Moving Average* Lines which is arrange in such way the each line is set n periods and k bar shift from each other this n and k are fibonaci related with each other.

- This line are name and formulated as follows:
  1. *Alligator’s jaw or the blue line* - This line is set to 13 period smoothed moving average and shifted into the future by 8 bars.
  2. *Alligator’s teeth or the red line* - This line is set to 8 period smoothed moving average and shifted into the future by 5 bars.
  3. *Alligator’s lips or the green line* - This line is a 5-period smoothed moving average shifted into the future by 3 bars.

Let's Calculate it one by one.

In [14]:
# Typical_Price.
typical_price = (my_stocks['Close'] + my_stocks['Low'] + my_stocks['High']) / 3

my_stocks['typical_price'] = typical_price
my_stocks['typical_price']

Date
2020-01-01    120.916667
2020-01-02    122.716667
2020-01-03    126.050001
2020-01-06    124.483332
2020-01-07    124.783333
                 ...    
2020-09-25     84.633331
2020-09-28     87.616666
2020-09-29     87.766668
2020-09-30     86.816668
2020-10-01     87.866666
Name: typical_price, Length: 189, dtype: float64

In [15]:
# Create Function for Moving Average 
def SMA(candle_list, period, shift):

  typical_price = (candle_list['Low'] + candle_list['High']) / 2

  list_1 = ['Nan']
  if shift == 3:
    median_prices_list = list_1 * (shift-1)
    for i in range(period, len(typical_price) + shift):
      sma = sum(typical_price[i - period:i])
      median_prices_list.append(sma/period)
  elif shift == 5:
    median_prices_list = list_1 * (shift-2)
    for i in range(period, len(typical_price) + shift):
      sma = sum(typical_price[i - period:i])
      median_prices_list.append(sma/period)
  elif shift == 8:
    median_prices_list = list_1 * (shift-3)
    for i in range(period, len(typical_price) + shift):
      sma = sum(typical_price[i - period:i])
      median_prices_list.append(sma/period)
  return median_prices_list

### 1. Alligator’s jaw or the blue line:
 $$Alligator.jaw={MA(Typical Price, 13, 8)}$$

In [16]:
# Calculate Alligator_jaw
allig_jaw = SMA(my_stocks, 13, 8)

my_stocks['allig_jaw'] = allig_jaw

### 2. Alligator’s teeth or the red line

$$Alligator.teeth={MA(Typical Price, 8, 5)}$$

In [17]:
# Calculate Alligator_jaw
allig_teeth = SMA(my_stocks, 8, 5)

my_stocks['allig_teeth'] = allig_teeth

### 3. Alligator’s lip or the green line

$$Alligator.lip={MA(Typical Price, 5, 3)}$$

In [18]:
# Calculate Alligator_jaw
allig_lip = SMA(my_stocks, 5, 3)

my_stocks['allig_lip'] = allig_lip
my_stocks['allig_lip']

Date
2020-01-01        Nan
2020-01-02        Nan
2020-01-03    123.765
2020-01-06    124.335
2020-01-07    124.735
               ...   
2020-09-25     85.565
2020-09-28      86.02
2020-09-29       86.8
2020-09-30      69.93
2020-10-01     52.465
Name: allig_lip, Length: 189, dtype: object

In [19]:
# Plot three Alligator Lines using Plotly Candle Sticks
fig=go.Figure()

fig.add_trace(go.Scatter(x=my_stocks.index, y= my_stocks['allig_jaw'], line=dict(width=1.5, color = 'blue'), name = 'Alligator Jaw'))
fig.add_trace(go.Scatter(x=my_stocks.index, y=my_stocks['allig_teeth'], line=dict(width=1.5, color = 'red'), name = 'Alligator Teeth'))
fig.add_trace(go.Scatter(x=my_stocks.index, y= my_stocks['allig_lip'], line=dict(width=1.5, color = 'green'), name = 'Alligator Lip'))

fig.add_trace(go.Candlestick(x=my_stocks.index, open=my_stocks['Open'], high=my_stocks['High'], low=my_stocks['Low'], close=my_stocks['Close'], name = 'Market Data'))


fig.show()
